In [13]:
!pip install requests
!pip install beautifulsoup4
!pip install -U notebook-as-pdf
!pip install pyppeteer

Requirement already up-to-date: notebook-as-pdf in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (0.4.0)


In [2]:
import requests 
from bs4 import BeautifulSoup 
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd

In [3]:
def get_cities(county_name):##This method assumes the county_name is in Michigan, and that the list of cities is part of an unordered list tag. 
    cities = []
    url = "https://geographic.org/streetview/usa/mi/"+county_name.lower()+"/index.html"
    #open with GET method 
    resp=requests.get(url) 
      
    #http_respone 200 means OK status 
    if resp.status_code==200: 
      
        # we need a parser,Python built-in HTML parser is enough . 
        soup=BeautifulSoup(resp.text,'html.parser')     
  
        # l is the list which contains all the cities
        l=soup.find("ul") 
      
        #Find all the elements of a, i.e anchor. This happens to correspond to the cities listed.
        for i in l.findAll("a"): 
            cities.append(i.text) 
    return cities

def get_all_cities(county_list):##This method returns a list of lists of city and county
    cities = []
    for county in county_list:
        for city in get_cities(county):
            cities.append((city,county))
    return cities

In [4]:
dma_tuples = get_all_cities(["Macomb","Oakland","Wayne"])

def create_location_dataframe(city_tuples):##Assumes we have a city,county tuple pair from Michigan
    location_dataframe = []
    geolocator = Nominatim(user_agent="detroit_explorer")
    for city,county in city_tuples:
        city_augmented = city+", MI"
        coordinates = geolocator.geocode(city_augmented)
        latitude = coordinates.latitude
        longitude = coordinates.longitude
        location_dataframe.append([city.title(),county.title(),latitude,longitude])
    location_dataframe = pd.DataFrame(location_dataframe,columns=["City","County","City Latitude","City Longitude"])
    return location_dataframe
dma_dataframe = create_location_dataframe(dma_tuples)

In [5]:
dma_dataframe = dma_dataframe[(dma_dataframe["City Latitude"]>=41) & (dma_dataframe["City Latitude"] <= 43) 
                              & (dma_dataframe["City Longitude"]>=-84) & (dma_dataframe["City Longitude"]<=-82)]

In [6]:
dma_dataframe

,City,County,City Latitude,City Longitude
0,Armada,Macomb,42.844196,-82.884372
1,Center Line,Macomb,42.485036,-83.027700
2,Clinton Township,Macomb,42.584852,-82.934824
3,Eastpointe,Macomb,42.468370,-82.955475
4,Fraser,Macomb,42.539202,-82.949365
5,Harrison Township,Macomb,42.587337,-82.810484
6,Macomb,Macomb,42.674119,-82.902901
7,Mount Clemens,Macomb,42.597256,-82.877975
8,New Baltimore,Macomb,42.681144,-82.736862
12,Romeo,Macomb,42.802808,-83.012987


In [ ]:
CLIENT_ID = '1N0NN5RFG2UBFYZMIOPR2GFAJDMNNK3LUNJLKXURFQ3Z1Q3Q' # your Foursquare ID
CLIENT_SECRET = 'KT5AM5W2ZLCGMHW11UK2MUBZ24Q0DLJCVOSRQRN1DJDRGCNB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # A default Foursquare API limit value

In [7]:
def getNearbyVenues(cities, counties, city_latitudes, city_longitudes, radius=1610):
    
    venues_list=[]
    for city, county, city_latitude, city_longitude in zip(cities, counties, city_latitudes,city_longitudes):
        ##print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            city_latitude, 
            city_longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            city,
            county,
            city_latitude, 
            city_longitude, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'County','City Latitude', 'City Longitude', 
                  'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [8]:
dma_venues = getNearbyVenues(cities=dma_dataframe['City'],
                             counties=dma_dataframe['County'],
                             city_latitudes=dma_dataframe['City Latitude'],
                             city_longitudes=dma_dataframe['City Longitude']
                            )
dma_venues

In [9]:
unique_venue_categories=list(dma_venues['Venue Category'].unique())
unique_venue_categories

['Restaurant',
 'Sandwich Place',
 'Bakery',
 'Pizza Place',
 'Bookstore',
 'Food',
 'Bar',
 'Park',
 'Trail',
 'Hardware Store',
 'American Restaurant',
 'Post Office',
 'Dessert Shop',
 'Butcher',
 'Italian Restaurant',
 'Pharmacy',
 'Arts & Crafts Store',
 'Chinese Restaurant',
 'Salon / Barbershop',
 'Rental Car Location',
 'Thrift / Vintage Store',
 'Diner',
 'Indian Restaurant',
 'Deli / Bodega',
 'Ice Cream Shop',
 'Auto Dealership',
 'Coffee Shop',
 'Fast Food Restaurant',
 'Convenience Store',
 'Optical Shop',
 'Discount Store',
 'Donut Shop',
 'Medical Supply Store',
 'Grocery Store',
 'Gas Station',
 'Thai Restaurant',
 'Intersection',
 'Automotive Shop',
 'Eastern European Restaurant',
 'Shopping Mall',
 'Electronics Store',
 'Gym',
 'Board Shop',
 'Liquor Store',
 'Marijuana Dispensary',
 'Snack Place',
 'Golf Course',
 'Cosmetics Shop',
 'Food Truck',
 'Home Service',
 'Flower Shop',
 'Event Service',
 'Record Shop',
 'Burger Joint',
 'Wings Joint',
 'Pet Store',
 'Event 

In [ ]:
restaurant_venue_categories = set(['Restaurant','Sandwich Place','Pizza Place','Food','Bar','American Restaurant','Italian Restaurant','Chinese Restaurant','Diner','Indian Restaurant',
                                  'Deli / Bodega','Fast Food Restaurant','Thai Restaurant','Eastern European Restaurant','Food Truck','Burger Joint','Wings Joint','Middle Eastern Restaurant',
                                  'Irish Pub','Sports Bar','Mexican Restaurant','Fried Chicken Joint','Asian Restaurant','Breakfast Spot','BBQ Joint', 'Cajun / Creole Restaurant','Pub',
                                  'Hot Dog Joint','Gluten-free Restaurant','Café','Sushi Restaurant','Greek Restaurant','Mediterranean Restaurant','Steakhouse','Seafood Restaurant','Bistro',
                                  'Cocktail Bar','Vietnamese Restaurant','Beer Garden','Gastropub','Taco Place','French Restaurant','Beer Bar','Korean Restaurant','Noodle House',
                                  'Japanese Restaurant','Mongolian Restaurant','Soup Place','Ethiopian Restaurant','Theme Restaurant','Whisky Bar','Polish Restaurant','Salad Place',
                                  'Yemeni Restaurant','Latin American Restaurant','Falafel Restaurant','Cuban Restaurant','Fish & Chips Shop','Tex-Mex Restaurant','Hungarian Restaurant',
                                  'Buffet'])
for category in restaurant_venue_categories:
    total = (dma_venues['Venue Category']==category).sum()
    print("Category: ", category," has ",total," results.")

In [12]:
dma_restaurant_venues = dma_venues[dma_venues['Venue Category'].isin(restaurant_venue_categories)]
dma_restaurant_venues.groupby(['County','Venue','Venue Latitude','Venue Longitude']).ngroups
dma_restaurant_venues.reset_index(drop=True,inplace=True)
dma_restaurant_venues

Category:  Food Truck  has  6  results.
Category:  Irish Pub  has  2  results.
Category:  Hungarian Restaurant  has  1  results.
Category:  Vietnamese Restaurant  has  2  results.
Category:  Polish Restaurant  has  3  results.
Category:  Mediterranean Restaurant  has  15  results.
Category:  Taco Place  has  9  results.
Category:  Tex-Mex Restaurant  has  2  results.
Category:  Indian Restaurant  has  3  results.
Category:  Thai Restaurant  has  30  results.
Category:  Cocktail Bar  has  11  results.
Category:  Eastern European Restaurant  has  2  results.
Category:  Breakfast Spot  has  19  results.
Category:  BBQ Joint  has  16  results.
Category:  Sushi Restaurant  has  19  results.
Category:  French Restaurant  has  2  results.
Category:  Ethiopian Restaurant  has  1  results.
Category:  Mongolian Restaurant  has  1  results.
Category:  Salad Place  has  3  results.
Category:  Cajun / Creole Restaurant  has  3  results.
Category:  Restaurant  has  30  results.
Category:  Food  has 

,City,County,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Armada,Macomb,42.844196,-82.884372,Papa's,42.844017,-82.884001,Restaurant
1,Armada,Macomb,42.844196,-82.884372,SUBWAY,42.844247,-82.884455,Sandwich Place
2,Armada,Macomb,42.844196,-82.884372,Tivoli's Pizza,42.844254,-82.883684,Pizza Place
3,Armada,Macomb,42.844196,-82.884372,Chap's Food & Spirits,42.844008,-82.883720,Restaurant
4,Armada,Macomb,42.844196,-82.884372,Papa's of Armada Family Restaurant,42.843985,-82.883772,Food
...,...,...,...,...,...,...,...,...
1103,Wyandotte,Wayne,42.200662,-83.151016,Big Boy Restaurant,42.199199,-83.151626,American Restaurant
1104,Wyandotte,Wayne,42.200662,-83.151016,Brooklyn's,42.199880,-83.168524,Bar
1105,Wyandotte,Wayne,42.200662,-83.151016,Wendy’s,42.198147,-83.152044,Fast Food Restaurant
1106,Wyandotte,Wayne,42.200662,-83.151016,Henry's On The River,42.209764,-83.147036,Café
